In [ ]:
%run ../config/init.py

### Creating results folders

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'trimmomatic')
result_dir = working_dir(os.path.join(RESULTS, DATASET, 'alignments'))
sample_table_file = os.path.join(DATA, DATASET, 'sample_table.csv')
sample_table = pandas.read_csv(sample_table_file, keep_default_na=False)
sample_table.head()

### Creating Genome indexes if they don't exists

In [ ]:
if not os.path.exists(GENOME):
    working_dir(GENOME)
    !curl -o {GENOME_NAME}.tar.gz https://ftp.ncbi.nlm.nih.gov/pub/pm4ngs/resources/{GENOME_NAME}.tar.gz
    !tar xzfv {GENOME_NAME}.tar.gz --strip 1
    !rm {GENOME_NAME}.tar.gz
    working_dir(ALIGNER_INDEX)
    log_file = 'bwa_index.log'
    index_cmd = '{0} {1}/bwa/bwa-index.cwl --sequences {2} > index.log 2>&1 &'.format(CWLRUNNER, CWLTOOLS, GENOME_FASTA)
    run_command(index_cmd)

In [ ]:
if os.path.exists('index.log'):
    check_cwl_command_log('index.log')

### Processing samples

In [ ]:
log_file = 'alignment.log'
working_dir(result_dir)
alignment_yml = {
    'genome_prefix': os.path.basename(GENOME_FASTA),
    'threads': 10,
    'genome_index': {'class': 'Directory', 'path': ALIGNER_INDEX },    
    'readsquality': 30,
    'subsample_nreads': 200000,
    'reads': []
}

for i, r in sample_table.iterrows():
{% if cookiecutter.sequencing_technology == 'paired-end' %}
    files = r['file'].split('|')
    r1 = os.path.join(DATA, DATASET, files[0])
    r2 = os.path.join(DATA, DATASET, files[1])
    if not os.path.exists(files[0].replace('_1.fastq.gz', '_sorted.bam')):
        alignment_yml['reads'].append([
            {'class': 'File', 'path': r1},
            {'class': 'File', 'path': r2}])
{% else %}
    f = os.path.join(DATA, DATASET, r['file'])
    if not os.path.exists(r['file'].replace('.fastq.gz', '_sorted.bam')):
        alignment_yml['reads'].append([
            {'class': 'File', 'path': f}])
{% endif %}
if alignment_yml['reads']:
    write_to_yaml(alignment_yml, 'alignment.yml')  
    cmd_header = '{} {}/ChIP-Seq/chip-seq-alignment.cwl alignment.yml > {} 2>&1 &'.format(
        CWLRUNNER, CWLWORKFLOWS, log_file)
    run_command(cmd_header)

### Checking command output
Execute next cell until it prints: **Run completed**

In [ ]:
check_cwl_command_log(log_file)

## Generating BAM files per condition

Here we merge all BAM files generated for each condition in one single BAM file named [condition]_sorted.bam

These files will be used with the output of IDR for the differential binding analysis

In [ ]:
log_file = 'merge.log'

merge_yml = {
    'bams': [],
    'out_bam': []    
}

for c in sample_table['condition'].unique():
    if not os.path.exists(c + '_sorted.bam'):
        ids = sample_table[sample_table['condition'] == c]['sample_name']
        if len(ids) > 1:            
            bams = []
            for s in ids:
                s = os.path.join(result_dir, s + '_sorted.bam')
                bams.append({'class': 'File', 'path': s})
            if bams:
                merge_yml['bams'].append(bams)
                merge_yml['out_bam'].append(c + '_sorted.bam')

if merge_yml['bams']:
    write_to_yaml(merge_yml, 'merge.yml')
    cmd_header = '{} {}/File-formats/merge-bam-parallel.cwl merge.yml > {} 2>&1 &'.format(
        CWLRUNNER, CWLWORKFLOWS, log_file)
    run_command(cmd_header)

### Checking command output
Execute next cell until it prints: **Run completed**

In [ ]:
check_cwl_command_log(log_file)

### Generating pooled tagAlign from replicates

In [ ]:
log_suffix = 'R0.log'
log_files = []

for c in sample_table['condition'].unique():
    ids = sample_table[sample_table['condition'] == c]['sample_name']
    if len(ids) > 1:
        cmd_header = 'zcat '
        for s in ids:
            s = os.path.join(result_dir, s + '_sorted.tagAlign.gz')
            cmd_header = '{0} {1}'.format(cmd_header, s)
        run_command('{0} | gzip -n > {1}_R0.tagAlign.gz 2> {1}_{2}\n'.format(cmd_header, c , log_suffix))  
        log_files.append('{0}_{1}'.format(c , log_suffix))

if log_files:
    all_good = True
    for l in log_files:
        if os.stat(l).st_size != 0:
            print('Error in file: ' + l)
            all_good = False
    if all_good:
        print('Pooled tagAlign files created correctly')